In [1]:
import datetime
import time
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
import argparse
import imutils
import time
import dlib
import cv2
import os
from deepface import DeepFace
import pyttsx3
from quoteslist import anger_quotes, sadness_quotes
import random


In [2]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    C = dist.euclidean(eye[0], eye[3])

    ear = (A + B) / (2.0 * C)

    return ear


In [3]:
def final_ear(shape):
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]

    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)

    ear = (leftEAR + rightEAR) / 2.0
    return (ear, leftEye, rightEye)


In [4]:
def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))

    low_lip = shape[56:59]
    low_lip = np.concatenate((low_lip, shape[65:68]))

    top_mean = np.mean(top_lip, axis=0)
    low_mean = np.mean(low_lip, axis=0)

    distance = abs(top_mean[1] - low_mean[1])
    return distance

In [5]:
def getemotion(frame):
    try:
        face_analysis = DeepFace.analyze(frame,actions = ['emotion'])
        dominant_emotion = face_analysis[0]['dominant_emotion']
        #print(face_analysis[0])
        emotion_value = face_analysis[0]['emotion'][dominant_emotion]

        #print(emotion_value)
        if emotion_value >= 50:
            print('a major emotion')
            #time.sleep(3)
            return dominant_emotion
            
        else:
            print('no major emotion')
            #time.sleep(3)
            return False
        
    except:
        return False
    


In [6]:
def getstatus(frame):
    global COUNTER
    global EYE_AR_THRESH
    global EYE_AR_CONSEC_FRAMES
    global YAWN_THRESH
    sleepy = False
    yawn = False
    dominant_emotion = getemotion(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector.detectMultiScale(gray, scaleFactor=1.1, 
		minNeighbors=5, minSize=(30, 30),
		flags=cv2.CASCADE_SCALE_IMAGE)
    print(rects)
    for (x, y, w, h) in rects:
        rect = dlib.rectangle(int(x), int(y), int(x + w),int(y + h))
        
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        eye = final_ear(shape)
        ear = eye[0]
        leftEye = eye [1]
        rightEye = eye[2]

        distance = lip_distance(shape)

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        lip = shape[48:60]
        cv2.drawContours(frame, [lip], -1, (0, 255, 0), 1)

        if ear < EYE_AR_THRESH:
            COUNTER += 1
            #print('sewy')

            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                #print('oki')
                if sleepy == False:
                    sleepy = True
                    #t = Thread(target=alarm, args=('wake up sir',))
                    #t.deamon = True
                    #t.start()

                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        else:
            COUNTER = 0
            sleepy = False

        if (distance > YAWN_THRESH):
                cv2.putText(frame, "Yawn Alert", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                if yawn == False and saying == False:
                    yawn = True
                    #t = Thread(target=alarm, args=('take some fresh air sir',))
                    #t.deamon = True
                    #t.start()
        else:
            yawn = False

        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "YAWN: {:.2f}".format(distance), (300, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        print(yawn,sleepy)

        
        
    final_data = {'sleepy': sleepy,
                      'yawn': yawn,
                      'dominant emotion': dominant_emotion}
    print(final_data, 'kasgdagdajd')
    return final_data
    

In [7]:
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")    #Faster but less accurate
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

engine = pyttsx3.init()
#ap = argparse.ArgumentParser()
#ap.add_argument("-w", "--webcam", type=int, default=0,
                #help="index of webcam on system")
#args = vars(ap.parse_args())

EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 30
YAWN_THRESH = 20
sleepy = False
yawn = False
saying = False
COUNTER = 0
#vs = VideoStream(0).start()
cap = cv2.VideoCapture(0)
timesincelastreport = time.time()
while True:
    #frame = vs.read()
    ret, frame = cap.read()
    frame = imutils.resize(frame, width=450)
    status = getstatus(frame)
    
    #print(status)
    emotion = status.get('dominant emotion')
    sleepy = status.get('sleepy')
    yawn = status.get('yawn')
    cv2.imshow("Frame", frame)
    #key = cv2.waitKey(1) & 0xFF

    #print(COUNTER)

    other_emotion = False
    


    if emotion != False:
        if timesincelastreport+5 <= time.time():
            if emotion == 'angry':
                other_emotion = True
                engine.say('please calm down')
                engine.runAndWait()
                engine.say(random.choice(anger_quotes))
                engine.runAndWait()
                timesincelastreport = time.time()
                #time.sleep(5)
            elif emotion == 'sad':
                #time.sleep(0.5)

                other_emotion = True
                engine.say('dont be sad')
                engine.runAndWait()
                engine.say(random.choice(sadness_quotes))
                engine.runAndWait()
                timesincelastreport = time.time()
                #time.sleep(5)
            # elif emotion == 'fear':
                # other_emotion = True
                # engine.say('dont be scared')
                # engine.runAndWait()
                # time.sleep(5)
            #elif emotion != 'neutral':
                #other_emotion == True
                #time.sleep(5)
            else:
                pass

    if sleepy  == True and other_emotion == False and timesincelastreport+5 <= time.time():
        engine.say('feeling sleepy? drink some water or have a coffee')
        engine.runAndWait()
        sleepy = False
        yawn = False
        print('slpehf')
        timesincelastreport = time.time()
        #time.sleep(5)

        print(other_emotion)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    print(sleepy,yawn)
cap.release()
cv2.destroyAllWindows()

2023-12-25 21:22:25.731 Python[70172:2369434] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


()
{'sleepy': False, 'yawn': False, 'dominant emotion': False} kasgdagdajd
False False


Action: emotion: 100%|██████████| 1/1 [00:00<00:00,  5.88it/s]


a major emotion
[[215 103  57  57]]
False False
{'sleepy': False, 'yawn': False, 'dominant emotion': 'neutral'} kasgdagdajd
False False


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 37.88it/s]


...

a major emotion
[[156  52  88  88]]
False False
{'sleepy': False, 'yawn': False, 'dominant emotion': 'neutral'} kasgdagdajd
False False


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 40.35it/s]


a major emotion
[[159  51  85  85]]
False False
{'sleepy': False, 'yawn': False, 'dominant emotion': 'neutral'} kasgdagdajd
False False


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 39.76it/s]


a major emotion
[[161  48  85  85]]
False False
{'sleepy': False, 'yawn': False, 'dominant emotion': 'neutral'} kasgdagdajd
